In [1]:
# 1 .First open the terminal and run this command
# sudo apt-get install build-essential g++ clang

# 2. Install LlamaCpp i face lots of issue but this 0.2.77 version is work fine
# !pip install llama-cpp-python==0.2.77


!pip install openai tiktoken  langchain langchain-community GitPython gpt4all llama-cpp-python==0.2.77  langchain-qdrant -q

In [2]:
import os
from git import Repo, GitCommandError
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

repo_url = "https://github.com/SuyodhanJ6/LLM-workshop-2024"
repo_path = "./suyo_repo"  # Changed to a relative path

try:
    # Check if the directory already exists
    if os.path.exists(repo_path):
        print(f"Directory {repo_path} already exists. Pulling latest changes...")
        repo = Repo(repo_path)
        origin = repo.remotes.origin
        origin.pull()
    else:
        print(f"Cloning repository to {repo_path}...")
        repo = Repo.clone_from(repo_url, repo_path)
    
    print("Repository cloned/updated successfully.")

    loader = GenericLoader.from_filesystem(
        repo_path,
        glob="**/*",
        suffixes=[".py"],
        parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
    )
    documents = loader.load()
    print(f"Number of documents loaded: {len(documents)}")

except GitCommandError as e:
    print(f"Git error occurred: {e}")
    print("Please check your internet connection and repository access.")
except Exception as e:
    print(f"An error occurred: {e}")

Directory ./suyo_repo already exists. Pulling latest changes...
Repository cloned/updated successfully.
Number of documents loaded: 5


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import Language

python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=2000,
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

26

In [4]:
texts[0]

Document(metadata={'source': 'suyo_repo/02_data/supplementary.py', 'language': <Language.PYTHON: 'python'>}, page_content='# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).\n# Source for "Build a Large Language Model From Scratch"\n#   - https://www.manning.com/books/build-a-large-language-model-from-scratch\n# Code: https://github.com/rasbt/LLMs-from-scratch\n\nimport torch\nimport tiktoken\nfrom torch.utils.data import Dataset, DataLoader\n\n\nclass GPTDatasetV1(Dataset):\n    def __init__(self, txt, tokenizer, max_length, stride):\n        self.input_ids = []\n        self.target_ids = []\n\n        # Tokenize the entire text\n        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})\n\n        # Use a sliding window to chunk the book into overlapping sequences of max_length\n        for i in range(0, len(token_ids) - max_length, stride):\n            input_chunk = token_ids[i:i + max_length]\n            target_chunk = token_ids[i + 1:

In [5]:
from langchain_community.embeddings import GPT4AllEmbeddings

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
gpt4all_embd = GPT4AllEmbeddings(model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs)

text = "This is a test document."

query_result = gpt4all_embd.embed_query(text)

doc_result = gpt4all_embd.embed_documents([text])

print(query_result)
print(doc_result)

/home/vivek/Documents/prashantPer/suyoxxx/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


[-0.0382995530962944, 0.1234196200966835, -0.028588155284523964, 0.05368538200855255, 0.008782053366303444, -0.039788126945495605, -0.0730057954788208, 0.04770145192742348, -0.0304920244961977, 0.05498405173420906, 0.08509133011102676, 0.036606401205062866, -0.005337036680430174, -0.002272902289405465, -0.06056099012494087, -0.027201388031244278, -0.011344251222908497, -0.042489923536777496, 0.009125489741563797, 0.1008230447769165, 0.07578961551189423, 0.06914955377578735, 0.009880030527710915, -0.0018218893092125654, 0.026144668459892273, 0.032788898795843124, -0.07189112901687622, 0.028421707451343536, 0.061703089624643326, -0.05254505202174187, 0.03358764573931694, 0.07441546767950058, 0.07543332874774933, 0.03536637872457504, 0.067121222615242, 0.010775715112686157, 0.0817243754863739, 0.016603022813796997, 0.032795388251543045, 0.036285385489463806, 0.002255137776955962, -0.09890527278184891, 0.005074367392808199, 0.050920628011226654, 0.009290034882724285, 0.024473417550325394, 

### Import required libraries

In [6]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryMemory
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

### Set up the GPT4AllEmbeddings

In [7]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs)


### Set up Qdrant vector store

In [8]:
# In-memory Qdrant setup
client = QdrantClient(":memory:")


In [9]:
client.create_collection(
    collection_name="autogen_collection1",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [10]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="autogen_collection1",
    embedding=embeddings,
)


In [11]:
type(texts[0])

langchain_core.documents.base.Document

In [12]:
vector_store.add_documents(texts)

['58042e88a53e47c89a6b86ac278f399b',
 '6523314393634e74b5259987c9e24b5b',
 '1fe1d91084ec43f39100d9ccac6907d1',
 '05f4db023d6943c18be48f04e2ca4e3f',
 '492845fcd25448ad8ee40c1cd9e65cdc',
 '49b177e31f56446998e6338011034693',
 'e984b5aa2f5548a295e5a6f99313f922',
 '002157d100b9462aae323362a3502559',
 '9f2231af78a14221a438d3aa8a29ecdc',
 '0cff688cf04d456cb31bc3a5578158f4',
 '130c4a8a1bdd43d8a45bcc2d9b776e4e',
 '7ce262908d0d48758b493ba2d1043664',
 '5b713b16dcce43f9823289503533278a',
 'ee9dbe0fd6c144dd9e181a167a4d8eab',
 '9ec295a26bd142bb9feac26ddf74aec2',
 'dfb96b9341b54bc9b4cd79db537edabd',
 'd17c8a729b67434fb280426cc9111ef2',
 'ab3c32db326d42af8174b08945b8a43e',
 'df518dc9cd0d4246a6493edabdd7e517',
 '8f185c81b54f42458e694786080eefa6',
 'b84faec44bfa492d9935c0b10cb1b487',
 'ea8440d41d964cd8b988d80d9d0a0876',
 '0492331e3ce54f219053cab7af74b92c',
 '717ab0ad67404c39838030915a71b332',
 'cf78d6d3cb104cb6bed8e21f69fa2707',
 '197600b90aff437585cf616d5e4cdad3']

### Create a retriever

In [13]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3},
)

In [14]:
retriever

VectorStoreRetriever(tags=['QdrantVectorStore', 'GPT4AllEmbeddings'], vectorstore=<langchain_qdrant.qdrant.QdrantVectorStore object at 0x7ab7ac047670>, search_type='mmr', search_kwargs={'k': 5})

In [15]:
template = """
Code Analysis Question: {question}

Analysis:
Let's break this down step by step:

1. Understand the question:
   [Brief restatement of the question]

2. Analyze the code or concept:
   [Detailed analysis of the code or concept in question]

3. Provide examples (if applicable):
   [Relevant code examples or illustrations]

4. Explain key points:
   [Explanation of important aspects or potential issues]

5. Best practices or improvements (if applicable):
   [Suggestions for better coding practices or improvements]

6. Conclusion:
   [Summary of the analysis and main takeaways]

Is there any specific part of this analysis you'd like me to elaborate on?
"""

prompt = PromptTemplate.from_template(template)



In [16]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

### Set up LlamaCpp model

In [17]:
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# llm = LlamaCpp(
#     model_path="codellama-13b-instruct.Q4_K_M.gguf",
#     n_ctx=5000,
#     n_gpu_layers=30,
#     n_batch=512,
#     f16_kv=True,
#     callback_manager=callback_manager,
#     verbose=True,
# )
n_gpu_layers = 1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="codellama-13b-instruct.Q4_K_M.gguf",
    n_ctx=5000,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)




llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from codellama-13b-instruct.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atten

llm_load_tensors:        CPU buffer size =  7500.96 MiB
....................................................................................................
llama_new_context_with_model: n_ctx      = 5024
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:        CPU KV buffer size =  3925.00 MiB
llama_new_context_with_model: KV self size  = 3925.00 MiB, K (f16): 1962.50 MiB, V (f16): 1962.50 MiB
llama_new_context_with_model:        CPU  output buffer size =     0.12 MiB
llama_new_context_with_model:        CPU compute buffer size =   442.32 MiB
llama_new_context_with_model: graph nodes  = 1286
llama_new_context_with_model: graph splits = 1
AVX = 1 | AVX_VNNI = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 |

In [27]:
llm_chain = prompt | llm 
question = "how to Preparing a dataset for supervised instruction finetuning?"
llm_chain.invoke({"question": question})
# from langchain_core.output_parsers import StrOutputParser

# chain = prompt | llm | StrOutputParser() 


In [26]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history", return_messages=True)  # prompt
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [24]:
question = "how to Preparing a dataset for supervised instruction finetuning?"
qa.invoke({"question": question})

Llama.generate: prefix-match hit




Preparing a dataset for supervised instruction finetuning can be done in the following steps.

1. Determine which task (e.g., "text-to-image" or "text-to-video") and which model (e.g., "GPT2" or "BART") to use for fine-tuning.



2. Acquire the dataset(s) needed to perform fine-tuning. The specific steps that need to be followed in order to acquire the dataset(s) needed to perform fine-tuning will depend on the specific task and model being used, as well as other factors such as the size of the dataset required to perform fine-tuning (e.g., "full" or "partial")).



3. Preprocess the acquired dataset(s)) to create a format that is suitable for use with fine-tuning. The specific steps that need to be followed in order to preprocess the acquired dataset(s)) will depend on the specific task and model being used, as well as other factors such as the size of the dataset required to perform fine-tuning (e.g., "full" or "partial")


llama_print_timings:        load time =   11170.36 ms
llama_print_timings:      sample time =     322.94 ms /   256 runs   (    1.26 ms per token,   792.70 tokens per second)
llama_print_timings: prompt eval time =   48495.75 ms /     2 tokens (24247.88 ms per token,     0.04 tokens per second)
llama_print_timings:        eval time =  447607.10 ms /   255 runs   ( 1755.32 ms per token,     0.57 tokens per second)
llama_print_timings:       total time =  452374.34 ms /   257 tokens
Llama.generate: prefix-match hit



The human asks how to prepare a dataset for supervised instruction finetuning. The AI respond

KeyboardInterrupt: 